<a href="https://colab.research.google.com/github/nachomendoza97/Data-Industrial/blob/main/Integraciones/APP_ABC_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
## Codigo de vinculacion de aplicaciones con plantillas de Google Sheets "Gestion de datos Industrial"
# 1. IMPORTACIÓN Y AUTENTICACIÓN
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from gspread.exceptions import SpreadsheetNotFound

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 2. CONEXIÓN A LA PLANILLA
id_planilla = "1aPsyywIl1GXf7R_GONxx8CCCqVS4gEenTWHtylUVUA8"

wb = None
try:
    wb = gc.open_by_key(id_planilla)
except SpreadsheetNotFound:
    print(f"Error: No se encontró la planilla con el ID '{id_planilla}'.")

# --- FUNCIÓN PARA PROCESAR ABC ---
def procesar_abc_desde_sheets():
    try:
        if wb is None:
            print("No se pudo conectar a la planilla, no se puede procesar ABC.")
            return

        hoja = wb.worksheet("INP_ABC")
        df = pd.DataFrame(hoja.get_all_records())

        # Lógica de Valor de Consumo y Clasificación
        df['Valor_Consumo'] = df['Costo Unitario'] * df['Demanda Anual']
        df = df.sort_values(by='Valor_Consumo', ascending=False).reset_index(drop=True)
        df['%_Acum_Items'] = (df.index + 1) / len(df)

        df['Categoria'] = df['%_Acum_Items'].apply(lambda x: 'A' if x <= 0.20 else ('B' if x <= 0.55 else 'C'))

        # --- EXPORTACIÓN PROTEGIDA ---
        res_hoja = wb.worksheet("Resultados")

        # 1. En lugar de clear() total, limpiamos solo el rango de datos (Columnas A a I)
        # Esto evita que se borre el gráfico que suele estar de la columna J en adelante.
        res_hoja.batch_clear(['A:I'])

        # 2. Preparamos los datos (Cabeceras + Valores)
        datos_a_subir = [df.columns.values.tolist()] + df.values.tolist()

        # 3. Actualizamos empezando desde la celda A1
        res_hoja.update('A1', datos_a_subir)

        print("✅ Análisis ABC finalizado. Los datos se actualizaron y el gráfico se mantuvo.")

    except Exception as e:
        print(f"Ocurrió un error durante el procesamiento: {e}")

# Ejecutar procesos
if wb is not None:
    procesar_abc_desde_sheets()

/tmp/ipython-input-2295740017.py:50: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  res_hoja.update('A1', datos_a_subir)


✅ Análisis ABC finalizado. Los datos se actualizaron y el gráfico se mantuvo.


In [4]:
print("Verificando el contenido de la hoja 'INP_ABC':")
if wb is not None:
    try:
        hoja_inp_abc = wb.worksheet("INP_ABC")
        df_inp_abc = pd.DataFrame(hoja_inp_abc.get_all_records())
        if not df_inp_abc.empty:
            print("Primeras 5 filas de la hoja 'INP_ABC':")
            display(df_inp_abc.head())
            print("Columnas encontradas en 'INP_ABC':")
            print(df_inp_abc.columns.tolist())
        else:
            print("La hoja 'INP_ABC' está vacía o no tiene registros.")
    except Exception as e:
        print(f"No se pudo leer la hoja 'INP_ABC': {e}")
else:
    print("No se pudo conectar a la planilla, por lo tanto no se puede verificar 'INP_ABC'.")


Verificando el contenido de la hoja 'INP_ABC':
Primeras 5 filas de la hoja 'INP_ABC':


,SKU,Descripcion,Costo Unitario,Precio de Venta,Demanda Anual,Proveedor
0,PRD010,Soporte para Monitor Ajustable,3600,7500,1500,TechSupply
1,PRD011,Hub USB 3.0 de 4 Puertos,900,2000,5000,GlobalConnect
2,PRD012,Filtro Anti-Luz Azul para Laptop,1200,3000,2200,VisionProtect
3,PRD013,Memoria USB 128GB Alta Velocidad,1800,4000,3000,DataStore
4,PRD014,Base Refrigeradora para Laptop Gaming,4300,9500,2500,GamerGear


Columnas encontradas en 'INP_ABC':
['SKU', 'Descripcion', 'Costo Unitario', 'Precio de Venta', 'Demanda Anual', 'Proveedor']
